In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
# Import stratifiedKFold
from sklearn.model_selection import KFold

In [23]:
from sklearn.preprocessing import StandardScaler
from pandas.api.types import is_numeric_dtype
def scale_data(data):
    scaler = StandardScaler()
    for col in data.columns:
        if is_numeric_dtype(data[col]):
            data[col] = scaler.fit_transform(data[[col]])
    return data

In [24]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
b = [1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 7.7, 8.8, 9.9, 12.0]
c = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
d = [object, object, object, object, object, object, object, object, object, object]
data = {'a': a, 'b': b, 'c': c, 'd': d}
df = pd.DataFrame(data)
df = scale_data(df)
df

,a,b,c,d
0,-1.566699,-1.518325,a,<class 'object'>
1,-1.218544,-1.187601,b,<class 'object'>
2,-0.870388,-0.856876,c,<class 'object'>
3,-0.522233,-0.526152,d,<class 'object'>
4,-0.174078,-0.195428,e,<class 'object'>
5,0.174078,0.135296,f,<class 'object'>
6,0.522233,0.466021,g,<class 'object'>
7,0.870388,0.796745,h,<class 'object'>
8,1.218544,1.127469,i,<class 'object'>
9,1.566699,1.758852,j,<class 'object'>


In [25]:
def cross_validate_model(model, X, y):
    cv_strat = KFold(n_splits=10, shuffle=True, random_state=42)
    cv_results = cross_validate(model, X, y, cv=cv_strat, scoring=['neg_root_mean_squared_error'], return_train_score=True)
    return cv_results

In [26]:
def average_cv_results(cv_results):
    avg_results = {}
    for key in cv_results.keys():
        avg_results[key] = cv_results[key].mean()
    return avg_results

In [34]:
# Load the data from pkl
cf = pd.read_pickle('../../data/processed/cf.pkl')
# Target is producto_1_cf
y = cf['producto_1_cf']
# Features are all columns except producto_1_cf
X = cf.drop(columns=['producto_1_cf'])
# Drop all columns which are not numeric
X = X.select_dtypes(include=['number'])
X.shape, y.shape

((152, 10), (152,))

In [51]:
# Perform cross validation
# Scale
X = scale_data(X)
model = RandomForestRegressor(n_estimators=30, random_state=42)
cv_results = cross_validate_model(model, X, y)
avg_results = average_cv_results(cv_results)
avg_results

{'fit_time': np.float64(0.018103981018066408),
 'score_time': np.float64(0.001117396354675293),
 'test_neg_root_mean_squared_error': np.float64(-251.98122802374883),
 'train_neg_root_mean_squared_error': np.float64(-98.80611799546435)}